This notebook contains the postprocessing of the results and visualization to compare with the results obtained of the semantic segmentation task on the toy data set in the corresponding paper.

# Utility functions
---

This section contains all auxilary functions required to compute the mean and the standard deviation of the metrics. It can be ignored except if new comparisons need to be made.

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties


## Compute Mean/Standard deviation/Maximum/Minimum

In [ ]:
def compute_mean_max_min_std_method(directory_results, method, seeds_list, metrics, directory_postprocessed_results, epochs):
    """Compute the mean, maximum, minimum, and standard deviation for a method over the different seeds for each metric provided.
    
    Parameters
    ----------
    directory_results : str
        String denoting the path to the directory where the results are stored.
    method : str
        String denoting the name of the method.
    seeds_list : list[int]
        List of integers denoting the pseudo-random seeds used during the experiments.
    metrics : list[str]
        List of strings denoting which metrics are considered. The metrics that are supported are: tra_sat_ratio, val_sat_ratio, tra_dice, 
        and val_dice. A numpy array of zeros will be stored if a different metric is passed to this function. It will be printed when an 
        unsupported metric is passed to this function.
    directory_postprocessed_results : str
        String denoting the path to the directory where the mean, maximum, minimum, and standard deviation arrays are stored.
    epochs : int
        Integer denoting the number of epochs that were done during training.
    
    Returns
    ----------
    
    """
    for current_metric in metrics:
        
        temp_mean_array = np.zeros([epochs, 1], dtype=np.float32)
        temp_max_array = np.zeros([epochs, 1], dtype=np.float32)
        temp_min_array = np.ones([epochs, 1], dtype=np.float32)
        temp_std_array = np.zeros([epochs, len(seeds_list)], dtype=np.float32)
        
        if current_metric == 'tra_sat_ratio' or current_metric == 'val_sat_ratio':
            for current_seed_index in range(0, len(seeds_list)):
                current_value = np.reshape(np.load(directory_results + method + str(seeds_list[current_seed_index]) + '/' + current_metric + '.npy'), [epochs, 1])

                temp_mean_array = temp_mean_array + current_value
                temp_max_array = np.maximum(temp_max_array, current_value)
                temp_min_array = np.minimum(temp_min_array, current_value)
                temp_std_array[:, current_seed_index] = np.reshape(current_value, [epochs,])

            temp_mean_array = temp_mean_array / len(seeds_list)
            temp_std_array = np.reshape(np.std(temp_std_array, axis=1), [epochs, 1])
            
        elif current_metric == 'tra_dice' or current_metric == 'val_dice':
            for current_seed_index in range(0, len(seeds_list)):
                current_value = np.reshape(np.mean(np.load(directory_results + method + str(seeds_list[current_seed_index]) + '/' + current_metric + '.npy')[:, :, -1], axis=1), [epochs, 1])
                
                temp_mean_array = temp_mean_array + current_value
                temp_max_array = np.maximum(temp_max_array, current_value)
                temp_min_array = np.minimum(temp_min_array, current_value)
                temp_std_array[:, current_seed_index] = np.reshape(current_value, [epochs,])

            temp_mean_array = temp_mean_array / len(seeds_list)
            temp_std_array = np.reshape(np.std(temp_std_array, axis=1), [epochs, 1])
        else:
            print('Metric: ' + str(current_metric) + ' is not supported. See the documentation to see which metrics are supported.')
                
        
        # save the results
        np.save(file=directory_postprocessed_results + method + current_metric + '_mean', arr=temp_mean_array)
        np.save(file=directory_postprocessed_results + method + current_metric + '_max', arr=temp_max_array)
        np.save(file=directory_postprocessed_results + method + current_metric + '_min', arr=temp_min_array)
        np.save(file=directory_postprocessed_results + method + current_metric + '_std', arr=temp_std_array)
        
    return

In [ ]:
def compute_satisfaction_ratio(directory_results, method, seeds_list, epochs):
    """ Compute the satisfaction ratio for the train and validation set from the number of constraints and the number of satisfied constraints.
    
    Parameters
    ----------
    directory_results : str
        String denoting the path to the directory where the results for each method is stored.
    method : str
        String denoting the method that was used during training.
    seeds_list : list[int]
        List of integers denoting the pseudo-random seeds used in the experiments.
    epochs : int
        Integer denoting the number of epochs that were done during training.
    
    Returns
    ----------
    
    """
    for i in range(0, len(seeds_list)):
        temp_tra_con = np.zeros([epochs, 1], dtype=np.float32)
        temp_tra_sat_con = np.zeros([epochs, 1], dtype=np.float32)
        temp_val_con = np.zeros([epochs, 1], dtype=np.float32)
        temp_val_sat_con = np.zeros([epochs, 1], dtype=np.float32)
        
        temp_tra_sat_ratio = np.zeros([epochs, 1], dtype=np.float32)
        temp_val_sat_ratio = np.zeros([epochs, 1], dtype=np.float32)
        
        temp_tra_con = np.reshape(np.sum(np.reshape(np.load(directory_results + method + str(seeds_list[i]) + '/' + 'tra_number_con' + '.npy'), [epochs, -1]), axis=1), [epochs, 1])
        temp_tra_sat_con = np.reshape(np.sum(np.reshape(np.load(directory_results + method + str(seeds_list[i]) + '/' + 'tra_sat_number_con' + '.npy'), [epochs, -1]), axis=1), [epochs, 1])
        temp_val_con = np.reshape(np.sum(np.reshape(np.load(directory_results + method + str(seeds_list[i]) + '/' + 'val_number_con' + '.npy'), [epochs, -1]), axis=1), [epochs, 1])
        temp_val_sat_con = np.reshape(np.sum(np.reshape(np.load(directory_results + method + str(seeds_list[i]) + '/' + 'val_sat_number_con' + '.npy'), [epochs, -1]), axis=1), [epochs, 1])
        
        temp_tra_sat_ratio = temp_tra_sat_con / temp_tra_con
        temp_val_sat_ratio = temp_val_sat_con / temp_val_con
        
        np.save(file=directory_results + method + str(seeds_list[i]) + '/' + 'tra_sat_ratio', arr=temp_tra_sat_ratio)
        np.save(file=directory_results + method + str(seeds_list[i]) + '/' + 'val_sat_ratio', arr=temp_val_sat_ratio)
    return 

In [ ]:
def compute_best_epoch_cggd(directory_results, method, seeds_list, tra=True):
    """ Compute the best epoch for cggd. This is the epoch with the highest satisfaction ratio because technically speaking we cannot use the labels.
    
    Parameters
    ----------
    directory_results : str
        String denoting the path to the directory where the results for each method is stored.
    method : str
        String denoting the method that was used during training.
    seeds_list : list[int]
        List of integers denoting the pseudo-random seeds used in the experiments.
    
    Returns
    ----------
    """
    if tra:
        for i in range(0, len(seeds_list)):
            current_tra_sat_ratio = np.load(directory_results + method + str(seeds_list[i]) + '/' + 'tra_sat_ratio.npy')
            current_index = np.reshape(np.argmax(current_tra_sat_ratio), [1,])

            np.save(file=directory_results + method + str(seeds_list[i]) + '/' + 'best_epoch_max_tra_sat', arr=current_index)
    else:
        for i in range(0, len(seeds_list)):
            current_tra_sat_ratio = np.load(directory_results + method + str(seeds_list[i]) + '/' + 'val_sat_ratio.npy')
            current_index = np.reshape(np.argmax(current_tra_sat_ratio), [1,])

            np.save(file=directory_results + method + str(seeds_list[i]) + '/' + 'best_epoch_max_tra_sat', arr=current_index)
    
    return 

In [ ]:
def convert_best_epoch_txt(directory_results, methods, seeds_list):
    """Convert the txt to a .npy file that contains the best epoch.
    Parameters
    ----------
    directory_results : str
        String denoting the path to the directory where the results for each method is stored.
    method : str
        String denoting the method that was used during training.
    seeds_list : list[int]
        List of integers denoting the pseudo-random seeds used in the experiments.
    
    Returns
    ----------
    """
    for current_method in methods:
        for i in range(0, len(seeds_list)):
            with open(directory_results + current_method + str(seeds_list[i]) + '/best_epoch.txt') as handle:
                current_best_epoch = handle.readlines()[0]
            np.save(directory_results + current_method + str(seeds_list[i]) + '/' + 'best_epoch.npy', arr=np.reshape(np.asarray(current_best_epoch), [1,]))
    return 

In [ ]:
def make_dataframe_mean_std(directory_results, directory_postprocessed_results, methods, metrics, seeds_list, epochs, custom_epoch=True, custom_epoch_base=False):
    """Compute the mean and standard deviation of the best epoch for the different methods and the different metrics.
    """
    result_array = np.zeros([len(methods), 2*len(metrics)], dtype=np.float32)
    
    for i in range(0, len(methods)):
        if methods[i] == 'cggd_size_centroid_' or methods[i] == 'cggd_box_prior_box_size_':
            if custom_epoch:
                for j in range(0, len(metrics)):
                    if metrics[j] == 'tra_dice' or metrics[j] == 'val_dice':
                        temp_array = np.zeros([len(seeds_list),], dtype=np.float32)
                        for k in range(0, len(seeds_list)):
                            current_value = np.reshape(np.mean(np.load(directory_results + methods[i] + str(seeds_list[k]) + '/' + metrics[j] + '.npy')[:, :, -1], axis=1), [epochs,])
                            current_best_epoch = np.load(directory_results + methods[i] + str(seeds_list[k]) + '/best_epoch_max_tra_sat.npy')

                            temp_array[k] = current_value[int(current_best_epoch)]

                        result_array[i, 2*j] = np.mean(temp_array)
                        result_array[i, 2*j+1] = np.std(temp_array)
                    elif metrics[j] == 'tra_sat_ratio' or metrics[j] == 'val_sat_ratio':
                        temp_array = np.zeros([len(seeds_list),], dtype=np.float32)
                        for k in range(0, len(seeds_list)):
                            current_value = np.reshape(np.load(directory_results + methods[i] + str(seeds_list[k]) + '/' + metrics[j] + '.npy'), [epochs,])
                            current_best_epoch = np.load(directory_results + methods[i] + str(seeds_list[k]) + '/best_epoch_max_tra_sat.npy')

                            temp_array[k] = current_value[int(current_best_epoch)]

                        result_array[i, 2*j] = np.mean(temp_array)
                        result_array[i, 2*j+1] = np.std(temp_array)
                    else:
                        print('Metric ' + metrics[j] + ' is not supported.')
            else:
                for j in range(0, len(metrics)):
                    if metrics[j] == 'tra_dice' or metrics[j] == 'val_dice':
                        temp_array = np.zeros([len(seeds_list),], dtype=np.float32)
                        for k in range(0, len(seeds_list)):
                            current_value = np.reshape(np.mean(np.load(directory_results + methods[i] + str(seeds_list[k]) + '/' + metrics[j] + '.npy')[:, :, -1], axis=1), [epochs,])
                            current_best_epoch = np.load(directory_results + methods[i] + str(seeds_list[k]) + '/best_epoch.npy')

                            temp_array[k] = current_value[int(current_best_epoch)]

                        result_array[i, 2*j] = np.mean(temp_array)
                        result_array[i, 2*j+1] = np.std(temp_array)
                    elif metrics[j] == 'tra_sat_ratio' or metrics[j] == 'val_sat_ratio':
                        temp_array = np.zeros([len(seeds_list),], dtype=np.float32)
                        for k in range(0, len(seeds_list)):
                            current_value = np.reshape(np.load(directory_results + methods[i] + str(seeds_list[k]) + '/' + metrics[j] + '.npy'), [epochs,])
                            current_best_epoch = np.load(directory_results + methods[i] + str(seeds_list[k]) + '/best_epoch.npy')

                            temp_array[k] = current_value[int(current_best_epoch)]

                        result_array[i, 2*j] = np.mean(temp_array)
                        result_array[i, 2*j+1] = np.std(temp_array)
                    else:
                        print('Metric ' + metrics[j] + ' is not supported.')
        else:
            if custom_epoch_base:
                for j in range(0, len(metrics)):
                    if metrics[j] == 'tra_dice' or metrics[j] == 'val_dice':
                        temp_array = np.zeros([len(seeds_list),], dtype=np.float32)
                        for k in range(0, len(seeds_list)):
                            current_value = np.reshape(np.mean(np.load(directory_results + methods[i] + str(seeds_list[k]) + '/' + metrics[j] + '.npy')[:, :, -1], axis=1), [epochs,])
                            current_best_epoch = np.load(directory_results + methods[i] + str(seeds_list[k]) + '/best_epoch_max_tra_sat.npy')

                            temp_array[k] = current_value[int(current_best_epoch)]

                        result_array[i, 2*j] = np.mean(temp_array)
                        result_array[i, 2*j+1] = np.std(temp_array)
                    elif metrics[j] == 'tra_sat_ratio' or metrics[j] == 'val_sat_ratio':
                        temp_array = np.zeros([len(seeds_list),], dtype=np.float32)
                        for k in range(0, len(seeds_list)):
                            current_value = np.reshape(np.load(directory_results + methods[i] + str(seeds_list[k]) + '/' + metrics[j] + '.npy'), [epochs,])
                            current_best_epoch = np.load(directory_results + methods[i] + str(seeds_list[k]) + '/best_epoch_max_tra_sat.npy')

                            temp_array[k] = current_value[int(current_best_epoch)]

                        result_array[i, 2*j] = np.mean(temp_array)
                        result_array[i, 2*j+1] = np.std(temp_array)
                    else:
                        print('Metric ' + metrics[j] + ' is not supported.')
            else:
                for j in range(0, len(metrics)):
                    if metrics[j] == 'tra_dice' or metrics[j] == 'val_dice':
                        temp_array = np.zeros([len(seeds_list),], dtype=np.float32)
                        for k in range(0, len(seeds_list)):
                            current_value = np.reshape(np.mean(np.load(directory_results + methods[i] + str(seeds_list[k]) + '/' + metrics[j] + '.npy')[:, :, -1], axis=1), [epochs,])
                            current_best_epoch = np.load(directory_results + methods[i] + str(seeds_list[k]) + '/best_epoch.npy')

                            temp_array[k] = current_value[int(current_best_epoch)]

                        result_array[i, 2*j] = np.mean(temp_array)
                        result_array[i, 2*j+1] = np.std(temp_array)
                    elif metrics[j] == 'tra_sat_ratio' or metrics[j] == 'val_sat_ratio':
                        temp_array = np.zeros([len(seeds_list),], dtype=np.float32)
                        for k in range(0, len(seeds_list)):
                            current_value = np.reshape(np.load(directory_results + methods[i] + str(seeds_list[k]) + '/' + metrics[j] + '.npy'), [epochs,])
                            current_best_epoch = np.load(directory_results + methods[i] + str(seeds_list[k]) + '/best_epoch.npy')

                            temp_array[k] = current_value[int(current_best_epoch)]

                        result_array[i, 2*j] = np.mean(temp_array)
                        result_array[i, 2*j+1] = np.std(temp_array)
                    else:
                        print('Metric ' + metrics[j] + ' is not supported.')
    
    header_list = []
    for i in metrics:
        header_list.append('mean ' + i)
        header_list.append('std ' + i)
        
    method_legend = {'fs_': 'Baseline', 'cggd_box_prior_box_size_': 'CGGD', 'logbarrier_box_prior_box_size_': 'Logbarrier', 'penalty_box_prior_box_size_': 'Penalty', 'cggd_size_centroid_': 'CGGD', 'logbarrier_size_centroid_': 'Logbarrier', 'penalty_size_centroid_': 'Penalty'}
    
    df_result = pd.DataFrame(result_array, columns=header_list, index=[method_legend[k] for k in methods])
    np.save(file=directory_postprocessed_results + 'mean_and_standard_deviation.npy', arr=result_array)
    return df_result

## Plot Mean/Standard deviation/Minimum/Maximum

In [ ]:
def plot_mean(directory_results, directory_postprocessed_results, methods, seeds_list, metrics, epochs):
    font = FontProperties()
    font.set_family('serif')
    mpl.rcParams['mathtext.fontset'] = 'custom'
    mpl.rcParams['mathtext.rm'] = 'Times New Roman'
    mpl.rcParams['mathtext.it'] = 'Times New Roman:italic'
    mpl.rcParams['mathtext.bf'] = 'Times New Roman:bold'
    colors_list = ['#377eb8', '#ff7f00', '#4daf4a', '#f781bf', '#a65628', '#984ea3', '#999999', '#e41a1c', '#dede00']
    metric_legend = {'tra_dice': 'dice loss train', 'val_dice': 'dice loss validation', 'tra_sat_ratio': 'satisfaction ratio train', 'val_sat_ratio': 'satisfaction ratio validation'}
    method_legend = {'fs_': 'Baseline', 'cggd_box_prior_box_size_': 'CGGD', 'logbarrier_box_prior_box_size_': 'Logbarrier', 'penalty_box_prior_box_size_': 'Penalty', 'cggd_size_centroid_': 'CGGD', 'logbarrier_size_centroid_': 'Logbarrier', 'penalty_size_centroid_': 'Penalty'}
    for i in range(0, len(metrics)):
        values_array = np.zeros([epochs, len(methods)], dtype=np.float32)
        for j in range(0, len(methods)):
            values_array[:, j] = np.reshape(np.load(directory_postprocessed_results + methods[j] + metrics[i] + '_mean.npy'), [epochs,])
        
        df_values = pd.DataFrame(values_array, index=range(1, epochs+1))
        fig, ax = plt.subplots(1, 1, figsize=(16,9))
        plt.title('Mean ' + metric_legend[metrics[i]])
        plt.axhline(y=0, color='black', linestyle='-')
        plt.axhline(y=1, color='black', linestyle='--')
        plt.xlim(0, epochs)
        plt.xlabel('Epoch')
        df_values.columns = [method_legend[k] for k in methods]
        df_values.plot(ax=ax, color=colors_list)
        plt.show()
    
    return 

In [ ]:
def plot_std(directory_results, directory_postprocessed_results, methods, seeds_list, metrics, epochs):
    font = FontProperties()
    font.set_family('serif')
    mpl.rcParams['mathtext.fontset'] = 'custom'
    mpl.rcParams['mathtext.rm'] = 'Times New Roman'
    mpl.rcParams['mathtext.it'] = 'Times New Roman:italic'
    mpl.rcParams['mathtext.bf'] = 'Times New Roman:bold'
    colors_list = ['#377eb8', '#ff7f00', '#4daf4a', '#f781bf', '#a65628', '#984ea3', '#999999', '#e41a1c', '#dede00']
    metric_legend = {'tra_dice': 'dice loss train', 'val_dice': 'dice loss validation', 'tra_sat_ratio': 'satisfaction ratio train', 'val_sat_ratio': 'satisfaction ratio validation'}
    method_legend = {'fs_': 'Baseline', 'cggd_box_prior_box_size_': 'CGGD', 'logbarrier_box_prior_box_size_': 'Logbarrier', 'penalty_box_prior_box_size_': 'Penalty', 'cggd_size_centroid_': 'CGGD', 'logbarrier_size_centroid_': 'Logbarrier', 'penalty_size_centroid_': 'Penalty'}
    for i in range(0, len(metrics)):
        values_array = np.zeros([epochs, len(methods)], dtype=np.float32)
        for j in range(0, len(methods)):
            values_array[:, j] = np.reshape(np.load(directory_postprocessed_results + methods[j] + metrics[i] + '_std.npy'), [epochs,])
        
        df_values = pd.DataFrame(values_array, index=range(1, epochs+1))
        fig, ax = plt.subplots(1, 1, figsize=(16,9))
        plt.title('Standard deviation ' + metric_legend[metrics[i]])
        plt.axhline(y=0, color='black', linestyle='-')
        plt.xlim(0, epochs)
        plt.xlabel('Epoch')
        df_values.columns = [method_legend[k] for k in methods]
        df_values.plot(ax=ax, color=colors_list)
        plt.show()
    
    return 

## Plot ratio between train and validation

In [ ]:
def plot_mean_ratio(directory_results, directory_postprocessed_results, methods, seeds_list, metrics, epochs):
    font = FontProperties()
    font.set_family('serif')
    mpl.rcParams['mathtext.fontset'] = 'custom'
    mpl.rcParams['mathtext.rm'] = 'Times New Roman'
    mpl.rcParams['mathtext.it'] = 'Times New Roman:italic'
    mpl.rcParams['mathtext.bf'] = 'Times New Roman:bold'
    colors_list = ['#377eb8', '#ff7f00', '#4daf4a', '#f781bf', '#a65628', '#984ea3', '#999999', '#e41a1c', '#dede00']
    metric_legend = {'tra_dice': 'dice loss train', 'val_dice': 'dice loss validation', 'tra_sat_ratio': 'satisfaction ratio train', 'val_sat_ratio': 'satisfaction ratio validation', 'dice': 'Dice loss', 'sat_ratio': 'Satisfaction ratio'}
    method_legend = {'fs_': 'Baseline', 'cggd_box_prior_box_size_': 'CGGD', 'logbarrier_box_prior_box_size_': 'Logbarrier', 'penalty_box_prior_box_size_': 'Penalty', 'cggd_size_centroid_': 'CGGD', 'logbarrier_size_centroid_': 'Logbarrier', 'penalty_size_centroid_': 'Penalty'}
    for i in range(0, len(metrics)):
        values_train_array = np.zeros([epochs, len(methods)], dtype=np.float32)
        values_val_array = np.zeros([epochs, len(methods)], dtype=np.float32)
        for j in range(0, len(methods)):
            values_train_array[:, j] = np.reshape(np.load(directory_postprocessed_results + methods[j] + 'tra_' +  metrics[i] + '_mean.npy'), [epochs,])
            values_val_array[:, j] = np.reshape(np.load(directory_postprocessed_results + methods[j] + 'val_' +  metrics[i] + '_mean.npy'), [epochs,])
        
        df_values = pd.DataFrame(values_train_array / values_val_array, index=range(1, epochs+1))
        fig, ax = plt.subplots(1, 1, figsize=(16,9))
        plt.title('Mean ratio between train and validation of ' + metric_legend[metrics[i]])
        plt.axhline(y=0, color='black', linestyle='-')
        plt.axhline(y=1, color='black', linestyle='--')
        plt.xlim(0, epochs)
        plt.ylim(0, 2)
        plt.xlabel('Epoch')
        df_values.columns = [method_legend[k] for k in methods]
        df_values.plot(ax=ax, color=colors_list)
        plt.show()
    
    return 


In [ ]:
def plot_std_ratio(directory_results, directory_postprocessed_results, methods, seeds_list, metrics, epochs):
    font = FontProperties()
    font.set_family('serif')
    mpl.rcParams['mathtext.fontset'] = 'custom'
    mpl.rcParams['mathtext.rm'] = 'Times New Roman'
    mpl.rcParams['mathtext.it'] = 'Times New Roman:italic'
    mpl.rcParams['mathtext.bf'] = 'Times New Roman:bold'
    colors_list = ['#377eb8', '#ff7f00', '#4daf4a', '#f781bf', '#a65628', '#984ea3', '#999999', '#e41a1c', '#dede00']
    metric_legend = {'tra_dice': 'dice loss train', 'val_dice': 'dice loss validation', 'tra_sat_ratio': 'satisfaction ratio train', 'val_sat_ratio': 'satisfaction ratio validation', 'dice': 'Dice loss', 'sat_ratio': 'Satisfaction ratio'}
    method_legend = {'fs_': 'Baseline', 'cggd_box_prior_box_size_': 'CGGD', 'logbarrier_box_prior_box_size_': 'Logbarrier', 'penalty_box_prior_box_size_': 'Penalty', 'cggd_size_centroid_': 'CGGD', 'logbarrier_size_centroid_': 'Logbarrier', 'penalty_size_centroid_': 'Penalty'}
    for i in range(0, len(metrics)):
        values_train_array = np.zeros([epochs, len(methods)], dtype=np.float32)
        values_val_array = np.zeros([epochs, len(methods)], dtype=np.float32)
        for j in range(0, len(methods)):
            values_train_array[:, j] = np.reshape(np.load(directory_postprocessed_results + methods[j] + 'tra_' +  metrics[i] + '_std.npy'), [epochs,])
            values_val_array[:, j] = np.reshape(np.load(directory_postprocessed_results + methods[j] + 'val_' +  metrics[i] + '_std.npy'), [epochs,])
        
        df_values = pd.DataFrame(values_train_array / values_val_array, index=range(1, epochs+1))
        fig, ax = plt.subplots(1, 1, figsize=(16,9))
        plt.title('Standard deviation ratio between train and validation of ' + metric_legend[metrics[i]])
        plt.axhline(y=0, color='black', linestyle='-')
        plt.xlim(0, epochs)
        plt.ylim(0, 2)
        plt.xlabel('Epoch')
        df_values.columns = [method_legend[k] for k in methods]
        df_values.plot(ax=ax, color=colors_list)
        plt.show()
    
    return 


## Compare convergence speed CGGD


In [ ]:
def compare_speed_mean(directory_postprocessed_results, directory_postprocessed_results_50, directory_postprocessed_results_10, metrics, seeds_list, epochs, epochs_50=None, epochs_10=None, train=True, val=True):
    font = FontProperties()
    font.set_family('serif')
    mpl.rcParams['mathtext.fontset'] = 'custom'
    mpl.rcParams['mathtext.rm'] = 'Times New Roman'
    mpl.rcParams['mathtext.it'] = 'Times New Roman:italic'
    mpl.rcParams['mathtext.bf'] = 'Times New Roman:bold'
    colors_list = ['#377eb8', '#ff7f00', '#4daf4a', '#f781bf', '#a65628', '#984ea3', '#999999', '#e41a1c', '#dede00']
    metric_legend = {'tra_dice': 'dice loss train', 'val_dice': 'dice loss validation', 'tra_sat_ratio': 'satisfaction ratio train', 'val_sat_ratio': 'satisfaction ratio validation', 'dice': 'Dice loss', 'sat_ratio': 'Satisfaction ratio'}
    method_legend = {'fs_': 'Baseline', 'cggd_box_prior_box_size_': 'CGGD', 'logbarrier_box_prior_box_size_': 'Logbarrier', 'penalty_box_prior_box_size_': 'Penalty', 'cggd_size_centroid_': 'CGGD', 'logbarrier_size_centroid_': 'Logbarrier', 'penalty_size_centroid_': 'Penalty'}
    size_legend = {'Full': 'Full', '50%': '50%', '10%': '10%'}
    sizes = ['Full', '50%', '10%']
    
    if epochs_50 is None:
        epochs_50 = epochs
    if epochs_10 is None:
        epochs_10 = epochs
    
    for i in range(0, len(metrics)):
        if train:
            values_array_full = np.zeros([epochs, ], dtype=np.float32)
            values_array_50 = np.zeros([int(epochs_50 / 2), ], dtype=np.float32)
            values_array_10 = np.zeros([int(epochs_10 / 10), ], dtype=np.float32)

            values_full = np.reshape(np.load(directory_postprocessed_results + 'cggd_size_centroid_' + 'tra_' +  metrics[i] + '_mean.npy'), [epochs,])
            values_50 = np.reshape(np.load(directory_postprocessed_results_50 + 'cggd_size_centroid_' + 'tra_' +  metrics[i] + '_mean.npy'), [epochs_50,])
            values_10 = np.reshape(np.load(directory_postprocessed_results_10 + 'cggd_size_centroid_' + 'tra_' +  metrics[i] + '_mean.npy'), [epochs_10,])

            for j in range(0, int(epochs_50 / 2)):
                values_array_50[j] = values_50[2*j]
            for j in range(0, int(epochs_10 / 10)):
                values_array_10[j] = values_10[10*j]

            values_dict = {}
            values_dict['Full'] = values_full
            values_dict['50%'] = values_array_50
            values_dict['10%'] = values_array_10
            
            df_values = pd.DataFrame.from_dict(values_dict, orient='index').transpose()
            fig, ax = plt.subplots(1, 1, figsize=(16,9))
            plt.title('Mean of ' + metric_legend[metrics[i]] + ' train for CGGD with different amounts of patients.')
            plt.axhline(y=0, color='black', linestyle='-')
            plt.axhline(y=1, color='black', linestyle='--')
            plt.xlim(0, max(epochs, int(epochs_50 / 2), int(epochs_10 / 10)))
            df_values.columns = [size_legend[k] for k in sizes]
            df_values.plot(ax=ax, color=colors_list)
            plt.show()
            
        if val:
            values_array_full = np.zeros([epochs, ], dtype=np.float32)
            values_array_50 = np.zeros([int(epochs_50 / 2), ], dtype=np.float32)
            values_array_10 = np.zeros([int(epochs_10 / 10), ], dtype=np.float32)

            values_full = np.reshape(np.load(directory_postprocessed_results + 'cggd_size_centroid_' + 'val_' +  metrics[i] + '_mean.npy'), [epochs,])
            values_50 = np.reshape(np.load(directory_postprocessed_results_50 + 'cggd_size_centroid_' + 'val_' +  metrics[i] + '_mean.npy'), [epochs_50,])
            values_10 = np.reshape(np.load(directory_postprocessed_results_10 + 'cggd_size_centroid_' + 'val_' +  metrics[i] + '_mean.npy'), [epochs_10,])

            for j in range(0, int(epochs_50 / 2)):
                values_array_50[j] = values_50[2*j]
            for j in range(0, int(epochs_10 / 10)):
                values_array_10[j] = values_10[10*j]

            values_dict = {}
            values_dict['Full'] = values_full
            values_dict['50%'] = values_array_50
            values_dict['10%'] = values_array_10
            
            df_values = pd.DataFrame.from_dict(values_dict, orient='index').transpose()
            fig, ax = plt.subplots(1, 1, figsize=(16,9))
            plt.title('Mean of ' + metric_legend[metrics[i]] + ' validation for CGGD with different amounts of patients.')
            plt.axhline(y=0, color='black', linestyle='-')
            plt.axhline(y=1, color='black', linestyle='--')
            plt.xlim(0, max(epochs, int(epochs_50 / 2), int(epochs_10 / 10)))
            df_values.columns = [size_legend[k] for k in sizes]
            df_values.plot(ax=ax, color=colors_list)
            plt.show()
        
        
    return

In [ ]:
def compare_speed_std(directory_postprocessed_results, directory_postprocessed_results_50, directory_postprocessed_results_10, metrics, seeds_list, epochs, epochs_50=None, epochs_10=None, train=True, val=True):
    font = FontProperties()
    font.set_family('serif')
    mpl.rcParams['mathtext.fontset'] = 'custom'
    mpl.rcParams['mathtext.rm'] = 'Times New Roman'
    mpl.rcParams['mathtext.it'] = 'Times New Roman:italic'
    mpl.rcParams['mathtext.bf'] = 'Times New Roman:bold'
    colors_list = ['#377eb8', '#ff7f00', '#4daf4a', '#f781bf', '#a65628', '#984ea3', '#999999', '#e41a1c', '#dede00']
    metric_legend = {'tra_dice': 'dice loss train', 'val_dice': 'dice loss validation', 'tra_sat_ratio': 'satisfaction ratio train', 'val_sat_ratio': 'satisfaction ratio validation', 'dice': 'Dice loss', 'sat_ratio': 'Satisfaction ratio'}
    method_legend = {'fs_': 'Baseline', 'cggd_box_prior_box_size_': 'CGGD', 'logbarrier_box_prior_box_size_': 'Logbarrier', 'penalty_box_prior_box_size_': 'Penalty', 'cggd_size_centroid_': 'CGGD', 'logbarrier_size_centroid_': 'Logbarrier', 'penalty_size_centroid_': 'Penalty'}
    size_legend = {'Full': 'Full', '50%': '50%', '10%': '10%'}
    sizes = ['Full', '50%', '10%']
    
    if epochs_50 is None:
        epochs_50 = epochs
    if epochs_10 is None:
        epochs_10 = epochs
    
    for i in range(0, len(metrics)):
        if train:
            values_array_full = np.zeros([epochs, ], dtype=np.float32)
            values_array_50 = np.zeros([int(epochs_50 / 2), ], dtype=np.float32)
            values_array_10 = np.zeros([int(epochs_10 / 10), ], dtype=np.float32)

            values_full = np.reshape(np.load(directory_postprocessed_results + 'cggd_size_centroid_' + 'tra_' +  metrics[i] + '_std.npy'), [epochs,])
            values_50 = np.reshape(np.load(directory_postprocessed_results_50 + 'cggd_size_centroid_' + 'tra_' +  metrics[i] + '_std.npy'), [epochs_50,])
            values_10 = np.reshape(np.load(directory_postprocessed_results_10 + 'cggd_size_centroid_' + 'tra_' +  metrics[i] + '_std.npy'), [epochs_10,])

            for j in range(0, int(epochs_50 / 2)):
                values_array_50[j] = values_50[2*j]
            for j in range(0, int(epochs_10 / 10)):
                values_array_10[j] = values_10[10*j]

            values_dict = {}
            values_dict['Full'] = values_full
            values_dict['50%'] = values_array_50
            values_dict['10%'] = values_array_10
            
            df_values = pd.DataFrame.from_dict(values_dict, orient='index').transpose()
            fig, ax = plt.subplots(1, 1, figsize=(16,9))
            plt.title('Standard deviation of ' + metric_legend[metrics[i]] + ' train for CGGD with different amounts of patients.')
            plt.axhline(y=0, color='black', linestyle='-')
            plt.axhline(y=1, color='black', linestyle='--')
            plt.xlim(0, max(epochs, int(epochs_50 / 2), int(epochs_10 / 10)))
            df_values.columns = [size_legend[k] for k in sizes]
            df_values.plot(ax=ax, color=colors_list)
            plt.show()
            
        if val:
            values_array_full = np.zeros([epochs, ], dtype=np.float32)
            values_array_50 = np.zeros([int(epochs_50 / 2), ], dtype=np.float32)
            values_array_10 = np.zeros([int(epochs_10 / 10), ], dtype=np.float32)

            values_full = np.reshape(np.load(directory_postprocessed_results + 'cggd_size_centroid_' + 'val_' +  metrics[i] + '_std.npy'), [epochs,])
            values_50 = np.reshape(np.load(directory_postprocessed_results_50 + 'cggd_size_centroid_' + 'val_' +  metrics[i] + '_std.npy'), [epochs_50,])
            values_10 = np.reshape(np.load(directory_postprocessed_results_10 + 'cggd_size_centroid_' + 'val_' +  metrics[i] + '_std.npy'), [epochs_10,])

            for j in range(0, int(epochs_50 / 2)):
                values_array_50[j] = values_50[2*j]
            for j in range(0, int(epochs_10 / 10)):
                values_array_10[j] = values_10[10*j]

            values_dict = {}
            values_dict['Full'] = values_full
            values_dict['50%'] = values_array_50
            values_dict['10%'] = values_array_10
            
            df_values = pd.DataFrame.from_dict(values_dict, orient='index').transpose()
            fig, ax = plt.subplots(1, 1, figsize=(16,9))
            plt.title('Standard deviation of ' + metric_legend[metrics[i]] + ' validation for CGGD with different amounts of patients.')
            plt.axhline(y=0, color='black', linestyle='-')
            plt.axhline(y=1, color='black', linestyle='--')
            plt.xlim(0, max(epochs, int(epochs_50 / 2), int(epochs_10 / 10)))
            df_values.columns = [size_legend[k] for k in sizes]
            df_values.plot(ax=ax, color=colors_list)
            plt.show()
        
        
    return

# Compute Mean/Standard deviation/Minimum/Maximum and best epoch CGGD

---

In [ ]:
# Directory results data set
directory_results = './results/toy/'
directory_results_10 = './results/toy_10/'
directory_results_50 = './results/toy_50/'

# The following methods are tested
full_sup = 'fs_'

cggd_size_centroid = 'cggd_size_centroid_'

logbarrier_size_centroid = 'logbarrier_size_centroid_'

penalty_size_centroid = 'penalty_size_centroid_'

list_methods = [full_sup, cggd_size_centroid, logbarrier_size_centroid, penalty_size_centroid]

# The following seeds were used for initializing the neural network
seeds_list = [50, 60, 70, 80]

# Directory where the postprocessed results should be stored
directory_postprocessed_results = './results/toy_postprocess/'

if not os.path.isdir(directory_postprocessed_results):
    os.makedirs(directory_postprocessed_results)
    
directory_postprocessed_results_10 = './results/toy_10_postprocess/'

if not os.path.isdir(directory_postprocessed_results_10):
    os.makedirs(directory_postprocessed_results_10)
    
directory_postprocessed_results_50 = './results/toy_50_postprocess/'

if not os.path.isdir(directory_postprocessed_results_50):
    os.makedirs(directory_postprocessed_results_50)


### full data set

In [ ]:
epochs = 50
# list metrics is the training dice loss, the validation dice loss, the train satisfaction ratio and the validation satisfaction ratio
list_metrics = ['tra_dice', 'val_dice', 'tra_sat_ratio', 'val_sat_ratio']

# before this can be done, the satisfaction ratio should be recomputed for each epoch.
for method in list_methods:
    compute_satisfaction_ratio_toy(directory_results=directory_results, method=method, seeds_list=seeds_list, epochs=epochs)

for method in list_methods:
    compute_mean_max_min_std_method(directory_results=directory_results, method=method, seeds_list=seeds_list, metrics=list_metrics, directory_postprocessed_results=directory_postprocessed_results, epochs=epochs)
    
compute_best_epoch_cggd_toy(directory_results=directory_results, method=cggd_size_centroid, seeds_list=seeds_list)
compute_best_epoch_cggd_toy(directory_results=directory_results, method=penalty_size_centroid, seeds_list=seeds_list)
compute_best_epoch_cggd_toy(directory_results=directory_results, method=logbarrier_size_centroid, seeds_list=seeds_list)
compute_best_epoch_cggd_toy(directory_results=directory_results, method=full_sup, seeds_list=seeds_list)
convert_best_epoch_txt(directory_results=directory_results, methods=list_methods, seeds_list=seeds_list)


### 50% of data set

In [ ]:
epochs = 50
# list metrics is the training dice loss, the validation dice loss, the train satisfaction ratio and the validation satisfaction ratio
list_metrics = ['tra_dice', 'val_dice', 'tra_sat_ratio', 'val_sat_ratio']

# before this can be done, the satisfaction ratio should be recomputed for each epoch.
for method in list_methods:
    compute_satisfaction_ratio_toy(directory_results=directory_results_50, method=method, seeds_list=seeds_list, epochs=epochs)

for method in list_methods:
    compute_mean_max_min_std_method(directory_results=directory_results_50, method=method, seeds_list=seeds_list, metrics=list_metrics, directory_postprocessed_results=directory_postprocessed_results_50, epochs=epochs)
    
compute_best_epoch_cggd_toy(directory_results=directory_results_50, method=cggd_size_centroid, seeds_list=seeds_list)
compute_best_epoch_cggd_toy(directory_results=directory_results_50, method=penalty_size_centroid, seeds_list=seeds_list)
compute_best_epoch_cggd_toy(directory_results=directory_results_50, method=logbarrier_size_centroid, seeds_list=seeds_list)
compute_best_epoch_cggd_toy(directory_results=directory_results_50, method=full_sup, seeds_list=seeds_list)
convert_best_epoch_txt(directory_results=directory_results_50, methods=list_methods, seeds_list=seeds_list)

### 10% of data set

In [ ]:
epochs = 50
# list metrics is the training dice loss, the validation dice loss, the train satisfaction ratio and the validation satisfaction ratio
list_metrics = ['tra_dice', 'val_dice', 'tra_sat_ratio', 'val_sat_ratio']

# before this can be done, the satisfaction ratio should be recomputed for each epoch.
for method in list_methods:
    compute_satisfaction_ratio_toy(directory_results=directory_results_10, method=method, seeds_list=seeds_list, epochs=epochs)

for method in list_methods:
    compute_mean_max_min_std_method(directory_results=directory_results_10, method=method, seeds_list=seeds_list, metrics=list_metrics, directory_postprocessed_results=directory_postprocessed_results_10, epochs=epochs)
    
compute_best_epoch_cggd_toy(directory_results=directory_results_10, method=cggd_size_centroid, seeds_list=seeds_list)
compute_best_epoch_cggd_toy(directory_results=directory_results_10, method=penalty_size_centroid, seeds_list=seeds_list)
compute_best_epoch_cggd_toy(directory_results=directory_results_10, method=logbarrier_size_centroid, seeds_list=seeds_list)
compute_best_epoch_cggd_toy(directory_results=directory_results_10, method=full_sup, seeds_list=seeds_list)
convert_best_epoch_txt(directory_results=directory_results_10, methods=list_methods, seeds_list=seeds_list)

# Show mean and standard deviation of best epoch
---

In [ ]:
# Directory results data set
directory_results = './results/toy/'
directory_results_10 = './results/toy_10/'
directory_results_50 = './results/toy_50/'

# The following methods are tested
full_sup = 'fs_'

cggd_size_centroid = 'cggd_size_centroid_'

logbarrier_size_centroid = 'logbarrier_size_centroid_'

penalty_size_centroid = 'penalty_size_centroid_'

list_methods = [full_sup, cggd_size_centroid, logbarrier_size_centroid, penalty_size_centroid]

# The following seeds were used for initializing the neural network
seeds_list = [50, 60, 70, 80]

# Directory where the postprocessed results should be stored
directory_postprocessed_results = './results/toy_postprocess/'

if not os.path.isdir(directory_postprocessed_results):
    os.makedirs(directory_postprocessed_results)
    
directory_postprocessed_results_10 = './results/toy_10_postprocess/'

if not os.path.isdir(directory_postprocessed_results_10):
    os.makedirs(directory_postprocessed_results_10)
    
directory_postprocessed_results_50 = './results/toy_50_postprocess/'

if not os.path.isdir(directory_postprocessed_results_50):
    os.makedirs(directory_postprocessed_results_50)
    

### full data set

In [ ]:
epochs = 50
list_metrics = ['tra_dice', 'val_dice', 'tra_sat_ratio', 'val_sat_ratio']

df_mean_std = make_dataframe_mean_std(directory_results=directory_results, directory_postprocessed_results=directory_postprocessed_results, methods=list_methods, metrics=list_metrics, seeds_list=seeds_list, epochs=epochs, custom_epoch_base=custom_epoch_base)

df_mean_std.head()

### 50% of data set

In [ ]:
epochs = 50
list_metrics = ['tra_dice', 'val_dice', 'tra_sat_ratio', 'val_sat_ratio']

df_mean_std_50 = make_dataframe_mean_std(directory_results=directory_results_50, directory_postprocessed_results=directory_postprocessed_results_50, methods=list_methods, metrics=list_metrics, seeds_list=seeds_list, epochs=epochs, custom_epoch_base=custom_epoch_base)

df_mean_std_50.head()

### 10% of data set

In [ ]:
epochs = 50
list_metrics = ['tra_dice', 'val_dice', 'tra_sat_ratio', 'val_sat_ratio']

df_mean_std_10 = make_dataframe_mean_std(directory_results=directory_results_10, directory_postprocessed_results=directory_postprocessed_results_10, methods=list_methods, metrics=list_metrics, seeds_list=seeds_list, epochs=epochs, custom_epoch_base=custom_epoch_base)

df_mean_std_10.head()